<a href="https://colab.research.google.com/github/pseudope/2021_F_Level2/blob/main/Projects/2%EC%A1%B0/3i4k_1008_%EC%9E%84%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

import konlpy
from konlpy.tag import Okt
from keras.preprocessing.text import Tokenizer

from keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout
from keras.models import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### 데이터 다운로드

---

In [2]:
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt

--2021-10-07 15:45:40--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661060 (2.5M) [text/plain]
Saving to: ‘fci_train_val.txt’

fci_train_val.txt   100%[===================>]   2.54M  --.-KB/s    in 0.02s   

2021-10-07 15:45:40 (157 MB/s) - ‘fci_train_val.txt’ saved [2661060/2661060]

--2021-10-07 15:45:40--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 2

In [3]:
#데이터 불러오기

def read_data(filename):

    with open(filename, "r", encoding="UTF8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]

    return data

In [4]:

train_data = read_data("/content/fci_train_val.txt")


#X,y
X_train_data = [t[1] for t in train_data]
y_train_data = [int(t[0]) for t in train_data]



test_data = read_data("/content/fci_test.txt")


#X,y
X_test_data = [t[1] for t in test_data]
y_test_data = [int(t[0]) for t in test_data]

In [5]:
print(len(X_train_data))
print(len(y_train_data))
print(len(X_test_data))
print(len(y_test_data))

55134
55134
6121
6121


In [6]:
#불용어 사전 가져오기

df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
df[0] = df[0].apply(lambda x: x.strip())

stopwords = df[0].to_numpy()

print(stopwords)
print(len(stopwords))

['이' '있' '하' '것' '들' '그' '되' '수' '이' '보' '않' '없' '나' '사람' '주' '아니' '등' '같'
 '우리' '때' '년' '가' '한' '지' '대하' '오' '말' '일' '그렇' '위하' '때문' '그것' '두' '말하'
 '알' '그러나' '받' '못하' '일' '그런' '또' '문제' '더' '사회' '많' '그리고' '좋' '크' '따르' '중'
 '나오' '가지' '씨' '시키' '만들' '지금' '생각하' '그러' '속' '하나' '집' '살' '모르' '적' '월' '데'
 '자신' '안' '어떤' '내' '경우' '명' '생각' '시간' '그녀' '다시' '이런' '앞' '보이' '번' '나' '다른'
 '어떻' '여자' '개' '들' '사실' '이렇' '점' '싶' '말' '정도' '좀' '원' '잘' '통하' '소리' '놓']
98


### 02. 토큰화 및 인코딩 (augmentation 생략)

In [7]:
!pip install konlpy
!pip install soynlp

     |████████████████████████████████| 19.4 MB 129 kB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 448 kB 63.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 416 kB 28.4 MB/s 


In [9]:
okt = Okt()

#형태소 토큰화


X_train_new=[]
for sentence_new in X_train_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_train_new.append(temp_X)

X_test_new = [] 
for sentence_new in X_test_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_test_new.append(temp_X)

In [10]:
print(X_train_new[:5])
print(X_test_new[:5])

[['만화'], ['이치'], ['약'], ['사이'], ['짜다']]
[['연두색'], ['되다'], [], ['그걸루'], ['연구소']]


In [11]:
#정수 인코딩

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_new)

total_cnt = len(tokenizer.word_index)
vocab_size = total_cnt + 1

In [12]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train_new)
X_train = tokenizer.texts_to_sequences(X_train_new)
X_test = tokenizer.texts_to_sequences(X_test_new)

In [13]:
X_train[:5]

[[3906], [7779], [389], [568], [915]]

In [14]:
#15로 패딩

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 15
# 한국어 특성상 문장 끝이 중요하므로 오른쪽에 치우치게 패딩

X_train_pad_new = pad_sequences(X_train, maxlen = max_len)
X_test_pad_new = pad_sequences(X_test, maxlen = max_len)

print(X_train_pad_new)
print(len(X_train_pad_new))
print(len(X_test_pad_new))

[[   0    0    0 ...    0    0 3906]
 [   0    0    0 ...    0    0 7779]
 [   0    0    0 ...    0    0  389]
 ...
 [   0    0    0 ...    0  317   32]
 [   0    0    0 ...    0   20   66]
 [   0    0    0 ... 1075  110    1]]
55134
6121


In [15]:
from tensorflow.keras.utils import to_categorical
print(len(set(y_train_data)))
y_train = to_categorical(y_train_data,len(set(y_train_data)))
y_test = to_categorical(y_test_data,len(set(y_train_data)))


print(len(X_train_pad_new))
print(len(X_test_pad_new))
print(len(y_train))
print(len(y_test))

7
55134
6121
55134
6121


In [16]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test
print(y_valid_new[0])
print(len(X_valid_new))
print(len(y_valid_new))

[0. 0. 0. 1. 0. 0. 0.]
5514
5514


In [17]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test
print(y_valid_new[0])
print(len(X_valid_new))
print(len(y_valid_new))

[0. 1. 0. 0. 0. 0. 0.]
5514
5514


## Modeling 1

In [19]:
!pip install tensorflow-addons

from tensorflow_addons.metrics import F1Score

     |████████████████████████████████| 1.1 MB 41.1 MB/s 


In [20]:
f1 = F1Score(num_classes=7, name="F1")
weighted_f1 = F1Score(num_classes=7, average="weighted", name="Weighted_F1")

In [23]:
model = Sequential() 
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc", f1, weighted_f1])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2174208   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 903       
Total params: 2,306,695
Trainable params: 2,306,695
Non-trainable params: 0
_________________________________________________________________


In [25]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [26]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc', f1, weighted_f1])
history = model.fit(X_train_new, y_train_new, epochs=50, callbacks=[es, mc], batch_size=500, validation_data = (X_valid_new, y_valid_new))

Epoch 1/50
100/100 [==============================] - 19s 171ms/step - loss: 0.1266 - acc: 0.8797 - F1: 0.7997 - Weighted_F1: 0.8669 - val_loss: 0.1533 - val_acc: 0.8012 - val_F1: 0.7161 - val_Weighted_F1: 0.8011

Epoch 00001: val_acc improved from -inf to 0.80123, saving model to best_model.h5
Epoch 2/50
100/100 [==============================] - 16s 162ms/step - loss: 0.0847 - acc: 0.8937 - F1: 0.8413 - Weighted_F1: 0.8924 - val_loss: 0.1582 - val_acc: 0.7980 - val_F1: 0.7038 - val_Weighted_F1: 0.7935

Epoch 00002: val_acc did not improve from 0.80123
Epoch 3/50
100/100 [==============================] - 16s 161ms/step - loss: 0.0777 - acc: 0.9015 - F1: 0.8526 - Weighted_F1: 0.9006 - val_loss: 0.1591 - val_acc: 0.7896 - val_F1: 0.7064 - val_Weighted_F1: 0.7857

Epoch 00003: val_acc did not improve from 0.80123
Epoch 4/50
100/100 [==============================] - 16s 161ms/step - loss: 0.0720 - acc: 0.9070 - F1: 0.8622 - Weighted_F1: 0.9062 - val_loss: 0.1599 - val_acc: 0.7952 - val_

In [27]:
loaded_model = load_model('best_model.h5')

In [28]:
loaded_model.evaluate(X_test_new, y_test_new)

192/192 [==============================] - 2s 6ms/step - loss: 0.1514 - acc: 0.8026 - F1: 0.7107 - Weighted_F1: 0.8011


[0.15140102803707123,
 0.8026466369628906,
 array([0.9265905 , 0.79067326, 0.875619  , 0.7878104 , 0.47953215,
        0.6043956 , 0.51027393], dtype=float32),
 0.801064670085907]